In [ ]:
# Для вас представлена задача, сделанная на открытых данных.

# Вам нужно предсказать цену товара по другим характеристикам.

# Метрика качества RMSLE 

# np.sqrt(np.mean((np.log(y_test + 1) - np.log(y_hat + 1))**2))
# np.sqrt(np.mean(np.power(np.log1p(y_test)-np.log1p(y_hat), 2)))
# np.sqrt(mean_squared_log_error(y_test, predictions))

# can_buy - возможно ли купить этот товар,

# can_promote - возможность продвигать товар,

# category - категория товара,

# contacts_visible - видны ли контакты,

# date_created - таймстемп создания товара,

# delivery_available - возможность доставки,

# description - описание,

# fields - свойства товара,

# id - идентификатор продукта,

# images - id картинок,

# location - местрорасположение,

# mortgage_available - возможность ипотеки,

# name - навзаник,

# payment_available - возможность онлайн оплаты,

# price - цена,

# subcategory - подкатегория,

# subway - метро,

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pickle
import seaborn as sns
import datetime as dt
warnings.filterwarnings('ignore')

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)

from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import validation_curve
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


[nltk_data] Downloading package stopwords to /home/alekum/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
STOPWORDS = []
STOPWORDS.extend(stopwords.words('english'))
STOPWORDS.extend(stopwords.words('russian'))# import pymorphy2

# morph  = pymorphy2.MorphAnalyzer()
# will care only about stopwords, neither isalnum or is_digit
# morphy_tokenizer = lambda text: [morph.parse(w)[0].normal_form for w in text.split() if w not in STOPWORDS] 
def nltk_tokenizer(text):
    return [w for w in nltk.word_tokenize(text) if w.isalnum() and w not in STOPWORDS]

In [4]:
# np.sqrt(np.mean((np.log(y + 1) - np.log(y_hat + 1))**2))    
# np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y_hat), 2))) 
# np.sqrt(mean_squared_log_error(y, y_hat)) 
def RMLSE_SCORE(y, y_hat):
    return np.sqrt(mean_squared_log_error(y, y_hat)) 

# E - model. estimator 
# X - data
# y - target
def RMLSE_SCORER(E, X, y):
    return RMLSE_SCORE(y, E.predict(X))

In [5]:
# Get rid of from all non helpful things from the text: по, (текст какой-то в скобках), slashes and etc
import re
re_parentheses_trash_ripper = re.compile("[!#$%&'*+.,^_`|~:;]|\(.*?\)", flags=re.IGNORECASE)
re_slash_dash_replacer = re.compile("[-/[\\]]")
re_double_quotes_ripper = re.compile("\".*?\"")
re_html_escapers = re.compile("&\w+;")
re_punctuation_ripper = re.compile("[!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]", flags=re.IGNORECASE)

def erase_chars_by_pattern(_text, _re_compiled_pattern, _substitution=''):
    _text = re.sub(_re_compiled_pattern, 
                   _substitution, 
                   str(_text)).strip()
    return _text

def name_descr_preproc(df):
    # I am too far away from doing it more efficiently putting it into pipeline(loop) with line of the parameters
    # so, suck it up=)
    df['name'] = df['name'].map(lambda value: erase_chars_by_pattern(value, re_parentheses_trash_ripper))
    df['name'] = df['name'].map(lambda value: erase_chars_by_pattern(value, re_slash_dash_replacer, ' '))
    df['name'] = df['name'].map(lambda value: erase_chars_by_pattern(value, re_double_quotes_ripper))
    df['name'] = df['name'].map(lambda value: str(value).lower())
    
    df['description'] = df['description'].map(lambda value: erase_chars_by_pattern(value, re_slash_dash_replacer, ' '))
    df['description'] = df['description'].map(lambda value: erase_chars_by_pattern(value, re_punctuation_ripper))
    df['description'] = df['description'].map(lambda value: erase_chars_by_pattern(value, re_html_escapers))
    df['description'] = df['description'].map(lambda value: str(value).lower())
    df['description'] = df['description'].map(lambda value: ' '.join([word for word in value.split() if word.isalnum()]))
    
    return df

In [5]:
# The kernel dies at this point
# train_sample_pd = load_pd_from_pickle('../input/hackathon-sf-ml-3/train_hack.pckl')

# train4col - name descr price 
df1_train = pd.read_csv('train_4_col.csv.zip', sep='\t', header=0, index_col=0, compression='zip')

# train4col2 - cat subcat
df2_train = pd.read_csv('train_4_col_2.csv.zip', sep='\t', header=0, index_col=0, compression='zip')

# train4col3 - id fields
# df3 = pd.read_csv('../input/hsfml12/train_4_col_3.csv', sep='\t', header=0, index_col=0)

# test_3_col - name descr price
df1_test = pd.read_csv('test_3_col.csv.zip', sep='\t', header=0, index_col=0, compression='zip')

# test_3_col2 - cat subcat
df2_test = pd.read_csv('test_3_col_2.csv.zip', sep='\t', header=0, index_col=0, compression='zip')

In [6]:
np.any(np.isnan(df1_train['price']))

False

In [7]:
# train dataset
df1_train.set_index('id',drop=True, inplace=True)
df1_train['description'] = df1_train['description'].map(lambda v: 'Не указано' if str(v) in 'nan' or not str(v).strip() else str(v)).astype('U')
df1_train['name'] = df1_train['name'].astype('U')
df1_train['price'] = df1_train['price'] / 100
df2_train.set_index('id',drop=True, inplace=True)
# df3.set_index('id',drop=True, inplace=True)


# test_dataset
df1_test.set_index('id',drop=True, inplace=True)
df1_test['description'] = df1_test['description'].map(lambda v: 'Не указано' if str(v) in 'nan' or not str(v).strip() else str(v)).astype('U')
df1_test['name'] = df1_test['name'].astype('U')
df2_test.set_index('id',drop=True, inplace=True)

In [8]:
np.any(np.isnan(df1_train['price']))

False

In [9]:
# merge two datasets
df_train = pd.concat([df1_train, df2_train], axis=1)
df_test = pd.concat([df1_test, df2_test], axis=1)

In [10]:
df_test.head()

,name,description,category,subcategory
id,,,,
285ea2e9935ccdeb8378c6a5,Стремянка,"Стремянка трехсекционная,3×4",6,603
adfb73820bbb831257df6e95,Плита электрическая,Плита ЭВИ 5120. Работают 3 конфорки и духовка....,2,203
783025601c36202f633fc6a5,Диски R17 Rays MazdaSpeed Touring 5х114.3 7J,Все лоты привезены с площадок и аукционов Япон...,1,116
2f0cd2d2e15dc90afd847f95,Batman Archam Knight,Не указано,10,1009
5c23a37902855a20172845a5,Ледобур для зимней рыбалки,Отдаю ледобур времён с.с.с.р диаметр 130 мм бе...,11,1104


In [11]:
df_train.head()

,name,description,price,category,subcategory
id,,,,,
3edeb34cf93f490ff760af85,Сумка DG,Не указано,1999.0,9,914
c98febd50dad3cc0ffc86085,Комплект,8-12 лет,350.0,22,2202
ade01e13912a46a99134cc75,Пальтишко демисезонное,"На девочку 1,5 г,состояние хорошее",300.0,22,2204
ab3e6941c11304c1519aef75,Attipas,"Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",800.0,22,2209
252452a91c944a22c276d995,Жилет теплый,Не указано,500.0,22,2204


In [12]:
np.any(np.isnan(df_train['price']))

False

In [16]:
df_train = df_train.drop(df_train[df_train['price']<=0].index) # delete all values with zero or negative price

In [17]:
np.any(np.isnan(df_train['price']))

False

In [18]:
# pickle.dump(df_train, open('df_train_name_desc_cat_subcat.pckl', 'wb'))
# pickle.dump(df_test, open('df_test_name_desc_cat_subcat.pckl', 'wb'))

In [6]:
df_train = pickle.load(open('df_train_name_desc_cat_subcat.pckl', 'rb'))
df_test = pickle.load(open('df_test_name_desc_cat_subcat.pckl', 'rb'))

In [ ]:
# cv_name = pickle.load(open('cv_name.pckl', 'rb'))
# cv_desc = pickle.load(open('cv_desc.pckl', 'rb'))

In [7]:
cv_name = CountVectorizer(tokenizer=nltk_tokenizer, ngram_range=(1,1))
cv_desc = CountVectorizer(tokenizer=nltk_tokenizer, ngram_range=(1,1))

In [12]:
cv_name.fit(df_train['name'].append(df_test['name']))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function nltk_tokenizer at 0x7f3379e6a378>,
        vocabulary=None)

In [13]:
cv_desc.fit(df_train['description'].append(df_test['description']))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function nltk_tokenizer at 0x7f3379e6a378>,
        vocabulary=None)

In [14]:
cv_name.vocabulary_.__len__()

203886

In [15]:
cv_desc.vocabulary_.__len__()

523074

In [16]:
from sklearn.base import BaseEstimator, TransformerMixin
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, col_name):
        self.col_name = col_name
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.col_name]
    
class ColumnsDropper(BaseEstimator, TransformerMixin):
    def __init__(self, col_names):
        assert isinstance(col_names,list), 'col_names must be a list of exists columns in dataframe'
        assert all([True if type(c) is str else False for c in col_names]), 'col_name must be a string'
        self.col_names = col_names
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.drop(self.col_names, axis=1) #.values ???
        return X
    
def columns_indicies(_df, columns):
    return [_df.columns.get_loc(c) for c in columns]

In [17]:
from sklearn.decomposition import TruncatedSVD
truncate_svd = TruncatedSVD(n_components=8)
scaler = StandardScaler()

In [18]:
nd_union = FeatureUnion([
        ('name_cv', Pipeline([
                            ('selector', ColumnSelector('name')),
                            ('cv_name', cv_name),
                            ('best', truncate_svd)
        ]))
        ,
        ('descr_cv', Pipeline([
                             ('selector', ColumnSelector('description')),
                             ('cv_descr', cv_desc),
                             ('best', truncate_svd)
        ]))
    ])

In [19]:
cv_union = FeatureUnion([
                    ('text', Pipeline([
                            ('nd_union', nd_union),
                            ('truncate', truncate_svd)
                        ]))
                    ,
                    ('nontext', Pipeline([
                            ('column_dropper', ColumnsDropper(['name', 'description'])),
#                             ('scaler', scaler)
                        ]))
                ])

In [20]:
model = Pipeline([
    ('union', cv_union)
    ,   
    ('rfr', RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=2))  
    ])

In [21]:
from sklearn.model_selection import validation_curve
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedKFold

In [22]:
y = df_train['price']
X = df_train.drop(['price'], axis=1)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
X_train.shape, X_test.shape

((1190088, 4), (510039, 4))

In [14]:
# param_grid_model = {
#     # 'rfr__n_estimators' : [50,100],
#     'union__nontext__scaler__with_mean' : [True, False],
#     'union__name_validation_curvecv__best__n_components': [50, 80],
#     'union__descr_cv__best__n_components': [50, 100]
# }

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.model_selection import validation_curve, cross_val_score

In [48]:
scores = cross_val_score(model, X_train, y_train, cv=3, scoring=RMLSE_SCORER)

In [49]:
np.mean(scores) # << new model with doing truncate two times (15)

1.7488560961375033

In [42]:
np.mean(scores) # << new model with doing truncate two times

1.8634836042597165

In [19]:
np.mean(scores) # << 10 n_components for TruncateSVD 

1.6289023969421976

In [16]:
np.mean(scores) # << 20 n_components for TrucnateSVD

1.639253735970718

In [17]:
np.mean(scores) # << default n_components for TruncateSVD

1.5430660930186348

In [25]:
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(memory=None,
     steps=[('nd_union', FeatureUnion(n_jobs=1,
       transformer_list=[('name_cv', Pipeline(memory=None,
     steps=[('selector', ColumnSelector(col_name='name')), ('cv_name', CountVectorizer(analyzer='...estimators=50, n_jobs=2,
           oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [26]:
y_hat = model.predict(X_train)

ValueError: dimension mismatch

In [ ]:
RMLSE_SCORE(y_train, y_hat)

# GRIDSEARCHCV

In [30]:
grid_search = GridSearchCV(model, param_grid_model, cv=3, scoring=RMLSE_SCORER)

In [ ]:
grid_search.fit(X_train, y_train) # < - it takes over 118h w/o any result

In [ ]:
grid_search.grid_scores_

# ~ PLAYING WITH TRAIN DF - MODEL Cross Validation

In [ ]:
# cross_val_scores = cross_val_score(model2, X_train, y_train, scoring=RMLSE_SCORER, cv=5, n_jobs=1)

In [ ]:
# cross_val_scores.mean(), cross_val_scores.std()

In [ ]:
# y_hat = model.predict(X_test)

In [ ]:
# RMLSE_SCORE(y, y_hat)

# DO PRODUCTION PREDICTION

In [ ]:
# For Submission

In [ ]:
prediction = model.predict(df_test)

In [ ]:
preds = pd.DataFrame(data=predictions, columns = ['regressor'])

#generating a submission file
result = pd.concat([df_test[['id']], preds], axis=1)
result.set_index('id', inplace = True)
result.head()

In [ ]:
result.to_csv('submission.csv')